Scratchpad for the LangChain tutorial demos to compare/contrast LangChain and OpenAI implementations.

Note that the documentation was made with `gpt-3.5-turbo-0301`; this notebook will use the updated `gpt-3.5-turbo-0613` for posterity.

Time is calculated for each cell to attempt to measure LangChain overhead, but results are variable so analysis is not performed. Times kept for posterity.

In [1]:
import openai
from dotenv import load_dotenv

load_dotenv()

True

## Hello World

The output of the Hello World is slightly different compared to the documentation due to the updated model (it's more accurate!)

In [2]:
%%time

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to French. I love programming.")])

CPU times: user 1.06 s, sys: 149 ms, total: 1.21 s
Wall time: 2.06 s


AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

In [3]:
%%time

import openai

messages = [{"role": "user", "content": "Translate this sentence from English to French. I love programming."}]

response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0)
response["choices"][0]["message"]["content"]

CPU times: user 3.46 ms, sys: 1.16 ms, total: 4.62 ms
Wall time: 672 ms


"J'adore la programmation."

## Memory

In [4]:
%%time

from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "The following is a friendly conversation between a human and an AI. The AI is talkative and "
        "provides lots of specific details from its context. If the AI does not know the answer to a "
        "question, it truthfully says it does not know."
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

CPU times: user 5.24 ms, sys: 1.29 ms, total: 6.54 ms
Wall time: 961 ms


'Hello! How can I assist you today?'

In [5]:
%%time

import openai

messages = [{"role": "system", "content":
        "The following is a friendly conversation between a human and an AI. The AI is talkative and "
        "provides lots of specific details from its context. If the AI does not know the answer to a "
        "question, it truthfully says it does not know."}]

user_message = "Hi there!"
messages.append({"role": "user", "content": user_message})
response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, temperature=0)
assistant_message = response["choices"][0]["message"]["content"]
messages.append({"role": "assistant", "content": assistant_message})
print(assistant_message)

Hello! How can I assist you today?
CPU times: user 3.62 ms, sys: 1.63 ms, total: 5.25 ms
Wall time: 1.08 s
